In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [6]:
import pandas as pd

In [7]:
options = ChromeOptions()
service = ChromeService(executable_path=ChromeDriverManager().install())

In [20]:
total_title_ko = []
total_title_en = []
total_journal = []
total_date = []
total_abstract= []

In [21]:
browser = webdriver.Chrome(service=service, options=options)

for page in range(0,2170,10):
                  
    browser.get(f"https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znTitle%2C%E2%80%9C%EC%83%9D%ED%99%9C%EC%B2%B4%EC%9C%A1%E2%80%9D&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount={page}&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=")
    browser.implicitly_wait(10)





    for i in range(1,11): # len(search_result_wrap)
        paper = browser.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div[4]/div/div[2]/div/div[3]/div[2]/ul/li[{i}]/div[2]/p[1]/a')
        
        time.sleep(1)
        paper.click()
        time.sleep(2)

        # 논문명
        try:
            title = browser.find_element(By.CSS_SELECTOR, "h3.title").text.strip()
            
            if " = " in title:
                title = title.split(' = ')

            else:
                title = [title,None]
            
                
        except:
            title = [None,None]
        
        total_title_ko.append(title[0])
        total_title_en.append(title[1])
        
        
        # 학술지
        try:
            journal = browser.find_element(By.CSS_SELECTOR, "a.text.mgb7").text.strip()
            
        except:
            journal = None   
        
        total_journal.append(journal)
        
        # 발행년도
        try:
            date = browser.find_element(By.XPATH, '//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[5]/div/p').text.strip()
            
        except:
            date = None   
        
        total_date.append(date)
        
        # 초록
        try:
            all_abstract_part = browser.find_elements(By.CSS_SELECTOR, 'a.moreView')[1:4]
            for moreView in all_abstract_part:
                moreView.click()
                time.sleep(1)
            
            abstract_part = browser.find_element(By.XPATH, '//*[@id="additionalInfoDiv"]')
            abstract = abstract_part.text.split("\n")
            
            en_abstract = None
            for index, value in enumerate(abstract):
                if "다국어 초록" in value:
                    en_index = index
                    en_abstract = abstract[en_index + 1].strip()
            
            total_abstract.append(en_abstract)
        except:
            total_abstract.append(None)
        
        # if "다국어 초록" in 
        # en_abstract_part = all_abstract_part.find_element(By.CSS_SELECTOR, 'a.moreView')
        # abstract_click.click()
        # # abstract_click = abstract_part.find_element(By.CSS_SELECTOR, 'a.moreView')
        # # abstract_click.click()
        # time.sleep(1)

        # try:
        #     abstract = browser.find_element(By.XPATH, 'div:contains("다국어 초록")')
        #     abstract = abstract.find_element(By.TAG_NAME, 'p').text.strip()

        # except:
        #     abstract = None   
        
        # total_abstract.append(abstract)
        
        
        browser.back()
        time.sleep(1)

# abstract = browser.find_element(By.XPATH, '//*[@id="additionalInfoDiv"]/div/div[1]/a')
# abstract.click()
# time.sleep(2)


In [22]:
print(len(total_title_ko)) 
print(len(total_title_en))
print(len(total_journal))
print(len(total_date ))
print(len(total_abstract))

2170
2170
2170
2170
2170


In [23]:
df = pd.DataFrame({"Title_ko":total_title_ko, "Title_en":total_title_en, "Publised":total_date, "Journal":total_journal, "Abstract":total_abstract})

In [24]:
df.to_excel("./DataFrame_KR.xlsx", index=False)